In [20]:
import sys
!{sys.executable} -V

Python 3.12.0


In [21]:
!{sys.executable} -m pip install \
    pytube \
    pytubefix \
    pillow

In [22]:
from pytube import YouTube

# Display thumbnail

In [23]:
yt = YouTube('http://youtube.com/watch?v=2lAe1cqCOXo')
yt.title

'YouTube Rewind 2019: For the Record | #YouTubeRewind'

In [24]:
from IPython.display import Image

Image(url=yt.thumbnail_url, width=512)

# Retrieve captions

In [25]:
# The solution in the documentation to retrive captions is not working as of 10-Dec-2023
# This is a workaround

yt = YouTube('http://youtube.com/watch?v=2lAe1cqCOXo')
stream = yt.streams.first()
captions = yt.captions

# List all caption languages
captions.lang_code_index

{'ar': <Caption lang="Arabic" code="ar">,
 'zh-HK': <Caption lang="Chinese (Hong Kong)" code="zh-HK">,
 'zh-TW': <Caption lang="Chinese (Taiwan)" code="zh-TW">,
 'hr': <Caption lang="Croatian" code="hr">,
 'cs': <Caption lang="Czech" code="cs">,
 'da': <Caption lang="Danish" code="da">,
 'nl': <Caption lang="Dutch" code="nl">,
 'en': <Caption lang="English" code="en">,
 'en-GB': <Caption lang="English (United Kingdom)" code="en-GB">,
 'et': <Caption lang="Estonian" code="et">,
 'fil': <Caption lang="Filipino" code="fil">,
 'fi': <Caption lang="Finnish" code="fi">,
 'fr-CA': <Caption lang="French (Canada)" code="fr-CA">,
 'fr-FR': <Caption lang="French (France)" code="fr-FR">,
 'de': <Caption lang="German" code="de">,
 'el': <Caption lang="Greek" code="el">,
 'iw': <Caption lang="Hebrew" code="iw">,
 'hu': <Caption lang="Hungarian" code="hu">,
 'id': <Caption lang="Indonesian" code="id">,
 'it': <Caption lang="Italian" code="it">,
 'ja': <Caption lang="Japanese" code="ja">,
 'ko': <Capt

In [26]:
# Let's select english language and convert to JSON so we can access the data as a dictionary

import json

en_json_captions = yt.captions['en'].json_captions
print(json.dumps(en_json_captions, sort_keys=True, indent=4))

{
    "events": [
        {
            "dDurationMs": 940,
            "segs": [
                {
                    "utf8": "K-pop!"
                }
            ],
            "tStartMs": 10200
        },
        {
            "dDurationMs": 2800,
            "segs": [
                {
                    "utf8": "That is so awkward to watch."
                }
            ],
            "tStartMs": 13400
        },
        {
            "dDurationMs": 2080,
            "segs": [
                {
                    "utf8": "YouTube Rewind 2018."
                }
            ],
            "tStartMs": 16200
        },
        {
            "dDurationMs": 3520,
            "segs": [
                {
                    "utf8": "The most disliked video\nin the history of YouTube."
                }
            ],
            "tStartMs": 18480
        },
        {
            "dDurationMs": 2220,
            "segs": [
                {
                    "utf8": "In 2018, we ma

In [28]:
# Finally, let's acess each line of the captions and print them out

lines = []
for captionEvent in en_json_captions["events"]:
    start = captionEvent["tStartMs"]
    duration = captionEvent["dDurationMs"]
    end = start + duration
    for segment in captionEvent["segs"]:
        text = segment["utf8"]
        lines.append((start, end, text))

for line in lines[:5]:
    print(line)

(10200, 11140, 'K-pop!')
(13400, 16200, 'That is so awkward to watch.')
(16200, 18280, 'YouTube Rewind 2018.')
(18480, 22000, 'The most disliked video\nin the history of YouTube.')
(22780, 25000, 'In 2018, we made something\nyou didn’t like.')


In [29]:
# We could convert these to an .srt style, but this is currently broken in pytube, let's do it by hand
# First let's create a function to decompose our milliseconds into hours, minutes, seconds and milliseconds

def ms_to_hmsm(milliseconds):
    hours, milliseconds = divmod(milliseconds, 3.6e+6)
    minutes, milliseconds = divmod(milliseconds, 6e+4)
    seconds, milliseconds = divmod(milliseconds, 1e+3)
    return (int(hours), int(minutes), int(seconds), int(milliseconds))

In [30]:
# Then let's create a function to transform that result into a .srt timestamp (01:07:32,053)
# We need to add trailing zeroes for single digits

def hmsm_srt(hmsm):
    return(f"""{hmsm[0]:02d}:{hmsm[1]:02d}:{hmsm[2]:02d},{hmsm[3]:03d}""")

In [31]:
# Time to view our output, and store it as an srt file

filename = "sub.srt"

with open(filename, 'w', encoding="utf-8") as f:
    for index, line in enumerate(lines):
        timestamp1 = hmsm_srt(ms_to_hmsm(line[0]))
        timestamp2 = hmsm_srt(ms_to_hmsm(line[1]))
        print(f"""{index+1}\n{timestamp1} --> {timestamp2}\n{line[2]}\n""")
        f.write(f"""{index+1}\n{timestamp1} --> {timestamp2}\n{line[2]}\n""")

1
00:00:10,200 --> 00:00:11,140
K-pop!

2
00:00:13,400 --> 00:00:16,200
That is so awkward to watch.

3
00:00:16,200 --> 00:00:18,280
YouTube Rewind 2018.

4
00:00:18,480 --> 00:00:22,000
The most disliked video
in the history of YouTube.

5
00:00:22,780 --> 00:00:25,000
In 2018, we made something
you didn’t like.

6
00:00:25,100 --> 00:00:27,570
So in 2019, let’s see what
you DID like.

7
00:00:27,580 --> 00:00:28,980
Because you’re better at this
than we are.

8
00:00:45,110 --> 00:00:46,420
This is my outfit!

9
00:00:46,920 --> 00:00:48,160
What is it gonna be?

10
00:00:58,180 --> 00:00:59,550
You made these the

11
00:00:59,710 --> 00:01:01,390
MOST LIKED CREATOR VIDEOS
*of 2019, duh…

12
00:01:02,480 --> 00:01:04,180
Shane Dawson | Conspiracy Theories with Shane Dawson
1.9M

13
00:01:05,000 --> 00:01:07,690
A4 | ПОКУПАЮ ВСЁ ЧТО ТЫ МОЖЕШЬ УНЕСТИ ИЗ МАГАЗИНА !
2M

14
00:01:07,690 --> 00:01:10,440
Andymation | My BIGGEST Flipbook EVER
2.3M

15
00:01:10,930 --> 00:01:12,660
James Ch

In [51]:
# Now that we know how to generate a .srt format manually, we can move to the pytubefix library to do it for us

from pytubefix import YouTube

yt = YouTube('http://youtube.com/watch?v=2lAe1cqCOXo')

caption = yt.captions.get_by_language_code('en')
caption.save_captions("captions.srt")

C:\Users\Steffan\AppData\Local\Temp\ipykernel_14752\3079985519.py:7: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('en')


# Using playlists

In [42]:
# Let's move to pytubefix, a fork of pytube with a number of fixed issues

from pytubefix import Playlist
playlist = Playlist('https://www.youtube.com/playlist?list=PLzH6n4zXuckoUWpzSEpQNW6I8rXIzyi8w')

In [43]:
# We can extract information from the propertiees of the Playlist instance

print(f"\
Playlist title:\t\t{playlist.title}\n\
View count:\t\t{playlist.views:,}\n\
Playlist owner:\t\t{playlist.owner}\n\
Video count:\t\t{playlist.length:,}\n\
\n{playlist.description}\n\
")

Playlist title:		Pong, Python & Pygame
View count:		29,047
Playlist owner:		Computerphile
Video count:		4

Squash-Pong created in python by Dr Isaac Triguero



In [44]:
# Retrieve a list of video names and corresponding url

from itertools import zip_longest

titles = [video.title for video in playlist.videos]
views = [video.views for video in playlist.videos]
urls = playlist.video_urls

videos = list(zip(titles, urls, views))

for video in videos:
    print(f"{video[0]} - {video[1]} ({video[2]:,} views)")

Pong, Python & Pygame 00 - Computerphile - https://www.youtube.com/watch?v=JRLdbt7vK-E (89,593 views)
Pong, Python & PyGame 01 - Computerphile - https://www.youtube.com/watch?v=hHtb-Ohyfu8 (60,029 views)
Pong, Python & Pygame 10 - Computerphile - https://www.youtube.com/watch?v=Nk3Och0I4ZY (37,610 views)
Pong, Python & PyGame 11 - Computerphile - https://www.youtube.com/watch?v=VyrAVNoEf0g (36,951 views)


In [55]:
# Download the playlist

for video in playlist.videos:
    video.streams.get_highest_resolution().download(output_path="./downloads/")

In [58]:
from IPython.display import Video

Video("./downloads/Pong Python & Pygame 00 - Computerphile.mp4", width=512)

In [46]:
# We can also download the audio tracks

for video in playlist.videos:
    video.streams.get_audio_only().download(mp3=True, output_path="./downloads/")

# Using channels

In [9]:
# pytube is broken so we will use pytubefix, a fork

from pytubefix import Channel

In [10]:
channel = Channel("https://www.youtube.com/@Computerphile/")

In [11]:
channel.channel_name

'Computerphile'

In [53]:
for url in channel.video_urls[:3]:
    print(url)

https://www.youtube.com/watch?v=K62EMzueWwA
https://www.youtube.com/watch?v=_eS-nNnkKfI
https://www.youtube.com/watch?v=olgGZmkkooU
